In [0]:
import zipfile
import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from google.colab import drive, files
from PIL import Image
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation,Flatten
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LambdaCallback
from keras.utils import np_utils
%matplotlib inline

drive.mount('/content/gdrive',force_remount=True)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Using TensorFlow backend.


## Unzipping the file with the images

In [0]:
with zipfile.ZipFile('/content/gdrive/My Drive/Big Data/Projet/celeba-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/img/')

# Création de la base de données

In [0]:
samp = pickle.load(open('/content/gdrive/My Drive/Big Data/Projet/Listes sample/Male.pkl', 'rb'))

In [0]:
path = '/content/img/img_align_celeba/img_align_celeba/'
target = []
data = np.zeros((len(samp), 64, 64, 3)) #dimension of the images
count = 0
for file in os.listdir(path) :
  if file in samp :
    image = Image.open(path + file)
    image = image.resize((64, 64))
    data[count] = np.array(image)
    target.append(file)
    count += 1

In [0]:
all_attributes = pd.read_csv('/content/img/list_attr_celeba.csv')
male = all_attributes[['image_id','Male']]
male = male.loc[male[male['image_id'].isin(target) == True].index]
male = male.set_index('image_id')
male = male.loc[target]
male.head()

,Male
image_id,
007271.jpg,-1
067551.jpg,-1
139194.jpg,-1
034274.jpg,-1
101776.jpg,-1


# Creation of the train and test sets

In [0]:
y = np.array(male.Male)
y[y == -1] = 0

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.2)

In [0]:
print(X_train.shape, y_train.shape)

(16000, 64, 64, 3) (16000,)


# Creation of the NN

In [0]:
male_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 20, 20, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 18, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [0]:
conv_net = Sequential()

# convolution layer 1
conv_net.add(Conv2D(64, (3, 3), activation = 'relu', input_shape = (64, 64, 3)))
conv_net.add(MaxPooling2D(pool_size = (3, 3)))
conv_net.add(Dropout(0.5))

print(conv_net.output.shape)

# convolution layer 2
conv_net.add(Conv2D(128, (3, 3), activation = 'relu'))
conv_net.add(MaxPooling2D(pool_size = (3, 3)))
conv_net.add(Dropout(0.5))


print(conv_net.output.shape)

# fully connected
conv_net.add(Flatten())
conv_net.add(Dense(1028, activation = 'relu'))
conv_net.add(Dropout(0.2))
conv_net.add(Dense(512, activation = 'relu'))
conv_net.add(Dropout(0.3))
conv_net.add(Dense(512, activation = 'relu'))
conv_net.add(Dropout(0.3))
conv_net.add(Dense(128, activation = 'relu'))
conv_net.add(Dropout(0.2))
conv_net.add(Dense(1, activation = 'sigmoid'))

conv_net.compile(loss = 'binary_crossentropy', optimizer = 'adam', 
                 metrics = ['accuracy'])

(None, 20, 20, 64)
(None, 6, 6, 128)


## callbacks to save the best model

In [0]:
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.2,
                              patience = 1, min_lr = 0.001)

# Path where we will save our model
filepath = "/content/gdrive/My Drive/Big Data/models MALE/Male.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'val_accuracy',
                             verbose = 1, save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, reduce_lr]

In [0]:
history = conv_net.fit(X_train, y_train, batch_size = 128, epochs = 10, 
                       validation_data = (X_test, y_test), callbacks = callbacks) 
#total of 50 epochs

Epoch 1/10
125/125 [==============================] - ETA: 0s - loss: 0.1080 - accuracy: 0.9620
Epoch 00001: val_accuracy improved from -inf to 0.94850, saving model to /content/gdrive/My Drive/Big Data/models MALE/Male.hdf5
125/125 [==============================] - 65s 520ms/step - loss: 0.1080 - accuracy: 0.9620 - val_loss: 0.1544 - val_accuracy: 0.9485 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - ETA: 0s - loss: 0.0900 - accuracy: 0.9671
Epoch 00002: val_accuracy did not improve from 0.94850
125/125 [==============================] - 66s 525ms/step - loss: 0.0900 - accuracy: 0.9671 - val_loss: 0.1533 - val_accuracy: 0.9467 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - ETA: 0s - loss: 0.0930 - accuracy: 0.9652
Epoch 00003: val_accuracy did not improve from 0.94850
125/125 [==============================] - 63s 502ms/step - loss: 0.0930 - accuracy: 0.9652 - val_loss: 0.1497 - val_accuracy: 0.9438 - lr: 0.0010
Epoch 4/10
125/125 [============

## Load the saved model

In [0]:
male_model = load_model("/content/gdrive/My Drive/Big Data/Final notebooks and models/Male.hdf5")

In [0]:
accuracy = male_model.evaluate(X_test, y_test, verbose = 0)[1]
print(accuracy)

0.9507499933242798
